In [1]:
from ioMicro import *

### Map data

In [2]:
data_folder= r'S:\ZaneTrit_3_3_2023'

# map all the hybes
hybes =  glob.glob(data_folder+os.sep+'H*')
# map all the fovs
fovs = [os.path.basename(fl)for fl in glob.glob(hybes[0]+os.sep+'*.zarr')]
def get_Hi(fld): 
    try: return int(os.path.basename(fld)[1:]) 
    except: return -1
    
hybes = np.array(hybes)[np.argsort([get_Hi(hybe) for hybe in hybes ])]

In [22]:
def drift_correction(hybes,fov,save_file,iiref=None,redo=False,ssz=20):
    if iiref is None: iiref = len(hybes)//2
    dic_drift = {}
    if os.path.exists(save_file):
        dic_drift = pickle.load(open(save_file,'rb'))
    tags = [os.path.basename(hybe) for hybe in hybes]
    redo = ~np.all([tag in dic_drift for tag in tags])
    if redo:
        #Load the reference dapi image
        fl1 = hybes[iiref]+os.sep+fov
        im1 = read_im(fl1)
        ncols,sz,sx,sy = im1.shape
        im1dapi = np.array(im1[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
        ###Perform drift

        for hybe in tqdm(hybes):
            fl2 = hybe+os.sep+fov
            tag = os.path.basename(hybe)
            if not (tag in dic_drift):
                im2 = read_im(fl2)
                ncols,sz,sx,sy = im1.shape
                

                im2dapi = np.array(im2[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
                txyz,txyzs = get_txyz(im1dapi,im2dapi,sz_norm=30,sz = 500,nelems=5)
                dic_drift[tag]=[txyz,txyzs]
                #print(txyz,txyzs)
                pickle.dump(dic_drift,open(save_file,'wb'))

In [30]:
def fit(hybe,fov,analysis_folder=r'S:\ZaneTrit_3_3_2023_analysis',icol=0,redo=False,hth = 200):
    fl = hybe+os.sep+fov
    tag = os.path.basename(hybe)
    save_file=analysis_folder+os.sep+fov.split('.')[0]+'--'+tag+'_fits_icol'+str(icol)+'.npz'
    if not os.path.exists(save_file) or redo:
        im = read_im(fl)
        ncols,sz,sx,sy = im.shape
        #for icol in np.arange(ncols-1):
        imS = np.array(im[icol],dtype=np.float32) #GFP cy5 half
        #Subtract local background
        imS_ = norm_slice(imS, s=30)
        #fit the spots
        Xh = get_local_max(imS_,hth,im_raw=imS,delta=1,delta_fit=3)
        np.savez_compressed(save_file,Xh=Xh)

In [31]:
fit(hybes[7], fovs[30],analysis_folder=r'S:\ZaneTrit_3_3_2023_analysis',icol=0,redo=False,hth = 200)

In [32]:
hybe, fov = hybes[7], fovs[30]
tag = os.path.basename(hybe)
icol = 0
save_file=analysis_folder+os.sep+fov.split('.')[0]+'_fits_icol'+str(icol)+'.npz'
Xh = np.load(save_file)['Xh']

In [36]:
fl = hybe+os.sep+fov
im = read_im(fl)

In [37]:
import napari
v = napari.view_image(im[icol])

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [38]:
#v.add_points(Xh[:,:3])
h = Xh[:,-1]
size = np.clip(h/np.percentile(h,99.9),0,1)*4
v.add_points(Xh[:,:3],size=size,face_color=[0,0,0,0],edge_color='y')


<Points layer 'Points' at 0x2450d96c250>

In [23]:
fov = fovs[30]
analysis_folder = r'S:\ZaneTrit_3_3_2023_analysis'
save_file = analysis_folder+os.sep+fov.split('.')[0]+'_drift.pkl'
print(save_file)
drift_correction(hybes,fov,save_file,iiref=None,redo=False,ssz=20)

S:\ZaneTrit_3_3_2023_analysis\Conv_zscan__30_drift.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [03:51<00:00, 12.17s/it]


In [20]:
dic_drift = pickle.load(open(save_file,'rb'))

In [21]:
dic_drift

{'H_GFP': [array([  0,   5, -93]),
  [array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   4, -92], dtype=int64),
   array([  0,   6, -93], dtype=int64)]],
 'H1': [array([ 0, 99, 90]),
  [array([ 0, 98, 90], dtype=int64),
   array([ 0, 99, 91], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 98, 90], dtype=int64)]],
 'H2': [array([ 0, 22, 45]),
  [array([ 0, 22, 45], dtype=int64),
   array([ 0, 23, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64)]],
 'H3': [array([ 0, -7, 16]),
  [array([ 0, -7, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64)]]}